In [1]:
import json
import pandas as pd
import altair as alt
import scipy.stats as stats
import numpy as np

pd.set_option('display.max_colwidth', 300)
pd.set_option("display.max_columns", 30)
alt.data_transformers.disable_max_rows()

def subreddit_counts(data, sample_size):
    
    if len(data) <= sample_size:
        return pd.DataFrame({'num': [999], 'count': 0, 'prob': 0})
    
    sub = data.sample(sample_size)

    extracted = sub['body'].str.extractall(r'([ #$-][1-9])') #.reset_index().drop(columns=['level_0'])
    num_counts = extracted[0].str.replace(' |#|\$|-', '').value_counts()
    num_counts = num_counts.reset_index()
    num_counts.columns = ['num', 'count']
    
    num_counts['num'] = num_counts['num'].astype(int)
    
    empty = pd.DataFrame({'num': range(1, 10), 'empty': 0})
    
    num_counts = pd.merge(num_counts, empty, on='num', how='right').fillna(0)
    num_counts['count'] = num_counts['count'] + num_counts['empty']
    
    return num_counts[['num', 'count']]

def perform_ks_test(data, benford):
    benford['b_count'] = (benford['prob'] * data['count'].sum()).astype(int)
    m = pd.merge(data[['num', 'count']], benford[['num', 'b_count']], on='num')
    p = stats.ks_2samp(m['count'], m['b_count'])[1]

    return p

#### Define Benford's Law


In [2]:
benford_prob = [[1, 30.1],
                [2, 17.6],
                [3, 12.5],
                [4, 9.7],
                [5, 7.9],
                [6, 6.7],
                [7, 5.8],
                [8, 5.1],
                [9, 4.6]]

benford_prob = pd.DataFrame(benford_prob, columns=['num', 'prob'])
benford_prob['prob'] = benford_prob['prob'] / 100
benford_prob['dataset'] = 'benford_prob'

#### Randomly choose comments from December 2019
The number of available comments is very large so I broke them up into several files. A simple, and probably weak, attempt to remove bots is performed and then the subreddits are grouped and 1000 comments are randomly drawn.
If fewer than 1000 are returned then that subreddit is ignored. Of the comments that are returned, the first digit of numbers within the comments are returned after satisfying this regular expression `r'([ #$-][1-9])'`.
Each first digit occurance is then summed.

In [7]:
num_counts = []

data = pd.DataFrame()

for i in range(10):
    data = pd.concat([data,
                     pd.read_csv(f'../../../../data_randomsample_site/benfords_reddit/comments_with_numbers_2019_12_00000000000{i}.csv.gz', nrows=1000000)],)
    
# Remove some bots
data['body'] = data['body'].str.lower()
data = data[~data['body'].str.contains('i\'m a bot')]
data = data[~data['body'].str.contains('i am a bot')]
data = data[~(data['author'].apply(lambda x: x[-3:].lower() == 'bot'))]

# A specific bot that pops up often
data = data[data['author'] != 'transcribersofreddit']

# &amp;#32 shows up often in comments but it not actually a string that the user types (links or some other kind of markdown?)
data['body'] = data['body'].str.replace('&amp;#32', '')

num_counts = data.groupby(['subreddit']).apply(lambda x: subreddit_counts(x, 500)).reset_index().drop(columns={'level_1'})

num_counts = num_counts[num_counts['num'] < 999]
num_counts = num_counts.groupby(['subreddit', 'num']).agg(count=('count', 'sum')).reset_index()

# Convert counts into frequencies (Frequency label here is `prob`)
num_counts['total'] =  num_counts.groupby(['subreddit'])['count'].transform('sum')
num_counts['prob'] = num_counts['count'] / num_counts['total']

#### Save results

Leading digit counts

In [10]:
num_counts.to_csv('../data/comment_dist_2019_12_2020_11_29.tsv', '\t')

### Select comments for scrolling

In [11]:
sub = data.sample(1000, random_state=10000)
sub = sub.rename(columns={'author': 'user', 'body': 'comment'}).reset_index(drop=True)
sub = sub.drop(columns=['score'])

In [12]:
extracted = sub['comment'].str.extractall(r'([ #$-][1-9])'
                                         ).reset_index().drop(columns=['match']
                                                             ).rename(columns={'level_0': 'comment_ind', 0: 'num'})

extracted['num'] = extracted['num'].str.replace(' |#|\$|-', '')

extracted = extracted.groupby(['comment_ind', 'num']).agg(count=('num', 'count')).reset_index()

extracted = pd.pivot_table(extracted, index='comment_ind', columns='num', values='count'
              ).fillna(0).astype(int).reset_index()

extracted = extracted.set_index('comment_ind')

sub = pd.merge(sub, extracted, left_index=True, right_index=True)
num_cols = [str(i) for i in range(1, 10)]
sub[num_cols] = sub[num_cols].cumsum(axis=0)

Save selected comments

In [59]:
sub.to_csv('../data/1k_comments_2020_11_29.csv')

#### Read results

In [60]:
# num_counts = pd.read_csv('../data/comment_dist_2019_12_2020_11_29.tsv', '\t')

#### Reshape the data and perform a few statistical tests

In [13]:
wide_counts = pd.pivot_table(num_counts, index=['subreddit'], columns=['num'], values=['count']).reset_index()

count_cols = [str(n) + '_count' for n in range(1, 10)]
wide_counts.columns = ['subreddit'] + count_cols

wide_counts['total'] = wide_counts[count_cols].sum(axis=1)

probs = wide_counts[count_cols].divide(wide_counts['total'], axis=0)
probs.columns = [str(n) + '_prob' for n in range(1, 10)]

wide_counts = pd.concat([wide_counts, probs], axis=1)

wide_counts['log_likelihood'] = wide_counts[count_cols].apply(lambda x: stats.multinomial.logpmf(x.values, x.sum(), benford_prob['prob'].values), axis=1)
wide_counts['kl_divergence'] = wide_counts[count_cols].apply(lambda x: stats.entropy(x.values, benford_prob['prob'].values), axis=1)
wide_counts = wide_counts.sort_values('log_likelihood', ascending=False).reset_index(drop=True)

# Very few subreddits have exceptionally low log-likelihoods so for the sake of plotting them in the future, truncate those that are very low.
wide_counts['log_likelihood'] = wide_counts['log_likelihood'].apply(lambda x: x if x >= -250 else -250)

# KS test. Output isn't used for now.
ks_results = num_counts.groupby(['subreddit']).apply(lambda x: perform_ks_test(x, benford_prob)).reset_index()
ks_results = ks_results.rename(columns={0: 'p'})

# Reduce columns to only what's needed.
wide_counts = wide_counts[['subreddit', 'log_likelihood'] + [str(n) + '_prob' for n in range(1, 10)]]
wide_counts.columns = ['subreddit', 'log_likelihood'] + [str(n) for n in range(1, 10)]

#### Save the processed data

In [14]:
wide_counts.to_csv('../data/comment_dist_wide_2019_12_2020_12_05.tsv', '\t')

#### See what we're working with by plotting some of the curves

In [15]:
wide_counts = wide_counts.sort_values('log_likelihood', ascending=False).reset_index(drop=True)
# wide_counts = wide_counts.sort_values('kl_divergence').reset_index(drop=True)

select_subs = wide_counts.iloc[:10]['subreddit']
# select_subs = ['science', 'pics', 'personalfinance', 'politics']
sub_counts = num_counts[num_counts['subreddit'].isin(select_subs)]

selector = alt.selection_multi(fields=['subreddit'], bind='legend')

plot = alt.Chart(sub_counts).mark_line().encode(
    x=alt.X('num'),
    y=alt.Y('prob'),
    color=alt.Color('subreddit', sort=alt.EncodingSortField('log_likelihood')),
    opacity=alt.condition(selector, alt.value(1), alt.value(0))
    ).add_selection(selector
    ).properties(width=300, height=200)

benny_plot = alt.Chart(benford_prob).mark_line(strokeWidth=3).encode(
    x=alt.X('num'),
    y=alt.Y('prob'))
    

(plot + benny_plot).configure_axis(labelFontSize=14, titleFontSize=14)

alt.LayerChart(...)

#### Distribution of log-likelihoods

In [16]:
alt.Chart(wide_counts).mark_bar().encode(
    x=alt.X('log_likelihood', bin=alt.Bin(maxbins=100)),
    y=alt.Y('count()'))

alt.Chart(...)